<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
SQL: Subqueries
              
</p>
</div>

Data Science Cohort Live NYC Feb 2022
<p>Phase 1: Topic 6</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
   

Queries can get complex:
- Helpful/necessary to build a query out of sub-units
- These subunits are queries which might include:
    - groupbys/agg functions
    - selecting/filtering
    - etc.

<img src = "Images/sql_subquery.jpg">

Inner query is a **subquery**:

- Generates an intermediate table that stores results temporarily in memory.
- Subquery intermediate table can be used during execution of main query.

Subqueries can be used to great effect in a variety of circumstances:
    
- Selecting from a table created by a subquery 
    - Subquery in FROM statement
- Filtering on a subquery.
    - Subquery in a WHERE statement
- Filtering in a SELECT statement.


#### Subquery in FROM clause

First load our handy dandy flights database.

In [55]:
import sqlite3
import numpy as np
import pandas as pd

# connect to flights database
con = sqlite3.connect('data/flights.db')

Recall the various tables in flight database:

In [56]:
%%bash 

sqlite3 data/flights.db
.tables

airlines  airports  routes  


In [57]:
pd.read_sql("""

SELECT * 
FROM airports
LIMIT 2

""", con)

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby


#### Problem

Get the average altitude of the highest altitude airports in the United States, Russia, and China.

Useful to break this into steps with a subquery.

- First filter airports on country.
- Then group by country and aggregate.
- Get average of the altitudes of the resultant airports.

Step by step.

Can also control execution flow and number of operations in a query this way.

Step 1: Filter airports on relevant countries.

In [ ]:
sq1 = pd.read_sql("""

SELECT *

FROM

airports

WHERE country IN 
('United States', 'China', 'Russia')
     

""", con)
sq1

In [ ]:
pd.read_sql("""

SELECT * 
FROM airports
LIMIT 2

""", con)

In [ ]:
sq1

Step 2: Use filtered table (via subquery) and get max altitude airport by country 

Want: name, city, altitude of these airports

In [ ]:
sq2 = pd.read_sql("""

SELECT cf.name, cf.city, 
MAX(CAST(altitude as INT)) AS maxalt
     
FROM 

(SELECT *
FROM airports 
WHERE country IN 
("United States", "Russia", "China") ) AS cf

GROUP BY country

     

""", con)

- Subquery in parentheses for defining temporary table
- Aliasing subquery: 
    - good practice
    - ease of referencing subquery fields in main query

In [ ]:
sq2

Now get the mean altitude across the three countries:

In [ ]:
sq3 = pd.read_sql("""

SELECT AVG(max_altitude) 
AS max_altitude_avg
    
FROM 

(
SELECT name, 
     city,
     country,
     MAX(CAST(altitude as float)) 
     AS max_altitude

FROM 

(SELECT *
FROM airports 
WHERE country IN 
("United States", "Russia", "China") ) AS cf
GROUP BY cf.country
)

     

""", con)

In [ ]:
sq3

Two levels of subqueries gets the job done.

**Exercise**

- Get me the countries whose max altitude and min altitude airport have a differential greater than 2000 ft.

- Use a subquery in the FROM statement.

In [ ]:
sq3 = pd.read_sql("""



""", con)
sq3

#### Using derived tables in joins

- create set of tables from subqueries
- use these to create a new table with derived columns from subqueries. 

Let's take an example of this.

For each airport I want:
- the count of routes that leaves from an airport.
- the count of routes that arrive there.


Many ways to do this. But I'll make full use of subqueries and joins here.

In [ ]:
pd.read_sql("""

SELECT * 
     FROM routes
     LIMIT 5


""", con)

First calculate count of routes departing from each airport.


In [ ]:
departure_table = pd.read_sql("""

SELECT source, COUNT(*) as departure_count

FROM routes

GROUP BY source

     
""", con)

In [ ]:
departure_table = pd.read_sql("""

SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source
     
""", con)

In [ ]:
departure_table

In [ ]:
arrival_table = pd.read_sql("""

SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest
     
""", con)

In [ ]:
arrival_table

We can link these the arrival tables and destination tables together:
- Keep records where source and destination are in both tables.
- What operation?

INNER JOIN ON SUBQUERY RESULTS!

In [ ]:
%%capture full_result

pd.read_sql("""

SELECT dep.source AS airport_code,
        dep.departure_count,
        arriv.arrival_count 
        
FROM

(SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source) AS dep

INNER JOIN 

(SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest) AS arriv

ON dep.source = arriv.dest
     
""", con)

In [ ]:
full_result()

This result is a derived table telling us information for each airport and keeping relevant records.

- We can filter or aggregate on this derived table as we see fit.


- Get all airports with significance difference in:
    - number of routes originating and terminating at given airport.
    - dfference in departure and arrival route counts greater than 5.
- Order in descending order of this difference.

Help this hapless data science instructor out!

In [ ]:
%%capture filtered_result

pd.read_sql("""

SELECT dep.source AS airport_code,
        dep.departure_count,
        arriv.arrival_count
        
FROM

(SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source) AS dep

INNER JOIN 

(SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest) AS arriv

ON dep.source = arriv.dest


     
""", con)

In [ ]:
%%capture filtered_result

pd.read_sql("""

SELECT dep.source AS airport_code,
        dep.departure_count,
        arriv.arrival_count, 
        ABS(dep.departure_count - arrival_count) AS count_diff
        
FROM

(SELECT source, COUNT(source) as departure_count
     FROM routes
     
     GROUP BY source) AS dep

INNER JOIN 

(SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     
     GROUP BY dest) AS arriv

ON dep.source = arriv.dest

WHERE count_diff > 5
ORDER BY count_diff DESC
     
""", con)

In [ ]:
filtered_result()

**Exercise**

Get me the name, city, country and airport code of airports for countries with more than 10 active airlines companies

Hints:
- DISTINCT(name) will be useful.
- A join might be useful. Aliasing tables will be important here as well.
- Think of groupbys and aggregations and filtering on them.

In [ ]:
%%bash 

sqlite3 data/flights.db
.schema airlines

In [ ]:
%%bash 

sqlite3 data/flights.db
.schema airports

In [ ]:
# Get me the name, city, country and airport code of airports for countries 
# with more than 10 active airlines companies

# Let's have fun building this up. Step by step. Start with subquery.
pd.read_sql("""
SELECT ar.name, ar.code, ar.city, ar.country, ct
FROM 
( SELECT country, COUNT(DISTINCT name) AS ct
FROM airlines 
WHERE active = 'Y'
GROUP BY country
HAVING ct > 10 ) as inter INNER JOIN airports as ar
ON inter.country = ar.country

""", con)

- Complex as subqueries pile on. 
- There are ways to organize subqueries: common table expressions
- Will make code readable.

But first: other ways subqueries can be used.

#### Subqueries in the WHERE statement

- Can use the result of a subquery to filter another selection.
- Particularly useful when you want to filter on the results of:
    - groupbys, etc
    - results from other tables

I want all route records where the departure airport has a departure route count greater than 200.

In [ ]:

pd.read_sql('''


SELECT source, COUNT(source)
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 200
''', con)

In [ ]:
%%capture where_subquery1
pd.read_sql('''


SELECT *
            
FROM routes

WHERE source IN (SELECT source 
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 200) 
''', con)

In [ ]:
where_subquery1()

Subqueries made this selection possible. 


#### The Semijoin

- Using a subquery from one table to filter another table.

Example:

Get the airport name, code, country, and altitude for airports with more than 200 routes originating from that airport.

In [ ]:
%%capture semijoin
pd.read_sql('''

SELECT name, code, country, 
CAST(altitude AS int) AS altitde
FROM airports

WHERE code IN (
SELECT source
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 200 
)
''', con)

In [ ]:
semijoin()

**Exercise**

Get me routes (all columns) whose destination airport is in the America/New York time zone.

In [ ]:
%%bash

sqlite3 data/flights.db
.schema airports

In [ ]:
%%bash

sqlite3 data/flights.db
.schema airlines

In [ ]:
# help me out 
pd.read_sql('''
SELECT *
FROM routes
WHERE dest IN
(SELECT code 
FROM airports 
WHERE timezone = 'America/New_York')

''', con)

#### The antijoin

Just the negation of the semijoin:

In [ ]:
%%capture antijoin

pd.read_sql('''
SELECT *
FROM airports

WHERE NOT code IN (
SELECT source 
     FROM routes
     GROUP BY source
     HAVING COUNT(source) > 300 
)
''', con)

In [ ]:
antijoin()

#### Subqueries in the SELECT statement

Get altitude of airport side by side with average airport altitude in the respective country.

In [ ]:
%%capture select_subquery

pd.read_sql('''
SELECT a1.country,
name, 
CAST(altitude as int) AS altitude ,

(SELECT AVG(CAST(altitude as int))
FROM airports AS a2
WHERE a2.country == a1.country
) AS avg_alt

FROM airports AS a1


''', con)

Sometimes known as a **correlated subquery**.

- Inner select statement filters a2 by country matching a given row's country in a1. Takes average altitude.

- Does this for each row in a1. The filtering of a2 by country *correlated* with current row in a1.


**Correlated subqueries can be a bit slow**


In [ ]:
select_subquery()

#### Common Table Expressions (CTEs)

Useful for when:

- Multiple subqueries floating around.
- Reusing same subqueries over and over again in same statement.

Cleans code up, reduces error, makes readable.

CTE syntax:

WITH subquery1_name AS (subquery1 statement),
<br>
WITH subquery2_name AS (subquery2 statement), 
<br>
...., 
<br>
WITH subquery_n_name AS (subquery_n statement)
<br>
<br>
MAIN QUERY

In [ ]:
%%capture filtered_result

pd.read_sql("""

WITH s1 AS (
     SELECT source, COUNT(source) as departure_count
     FROM routes
     GROUP BY source),
     
     s2 AS ( 
     SELECT dest, COUNT(dest) as arrival_count
     FROM routes
     GROUP BY dest
     )


SELECT source, departure_count, arrival_count
FROM
s1 INNER JOIN s2
ON s1.source = s2.dest

     
""", con)

In [ ]:
filtered_result()

CTEs make the primary query easy to read.

- Subqueries can just be referenced by looking it up in the CTE.

In [ ]:
df3 = pd.read_sql("""

SELECT source, dest,
ap1.city AS source_city,
ap2.city AS dest_city,
ap1.country AS source_country, 
ap2.country AS dest_country,
CAST(ap1.altitude as float) AS alt,
max_alt

FROM routes
INNER JOIN 

(
SELECT * 
FROM
airports
WHERE country == 'United States') as ap1


INNER JOIN 

(
SELECT country, 
city, name, code, 
MAX(CAST(altitude as float)) AS max_alt

FROM airports 
GROUP BY country
)  as ap2

ON (routes.source == ap1.code AND routes.dest == ap2.code)

""", con)

In [137]:
df3 = pd.read_sql("""

SELECT source, dest,
ap1.city AS source_city,
ap2.city AS dest_city,
ap1.country AS source_country, 
ap2.country AS dest_country,
CAST(ap1.altitude as float) AS alt,
max_alt

FROM routes
INNER JOIN 

(
SELECT * 
FROM
airports
WHERE country == 'United States') as ap1


INNER JOIN 

(
SELECT country, 
city, name, code, 
MAX(CAST(altitude as float)) AS max_alt

FROM airports 
GROUP BY country
)  as ap2

ON (routes.source == ap1.code AND routes.dest == ap2.code)

""", con)